### Data Connection

data connection은 모델에 제공하려는 추가적인 non-parametic 지식을 검색하는 데 필요한 빌딩 블록을 의미한다. 이 아이디어는 5개의 주요 블록(원천 - 로드 - 변환 - 임베드 - 저장 - 검색)으로 이뤄지는 애플리케이션에 사용자별 데이터를 통합하는 일반적인 흐름을 다루기 위한 것이다.

#### Document loaders

document loader는 CSV, file directory, HTML, JSON, markdown, PDF 등 다양한 source로부터 문서를 읽어 들이는 일을 한다. document loader는 구성된 소스로부터 데이터를 문서로 loading하기 위한 `.load` method를 노출한다. 출력은 텍스트와 관련 메타데이터를 포함하는 `Document` 객체이다.

In [1]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [2]:
dot_data = pd.read_csv(
    '../data/dot_plot_2024_12.csv',
    index_col = 0
)

In [3]:
dot_data

,2024,2025,2026,2027,Longer run
Target rate,,,,,
4.750,NaN,NaN,NaN,NaN,NaN
4.625,4.0,NaN,NaN,NaN,NaN
4.500,NaN,NaN,NaN,NaN,NaN
4.375,15.0,1.0,NaN,NaN,NaN
4.250,NaN,NaN,NaN,NaN,NaN
4.125,NaN,3.0,NaN,NaN,NaN
4.000,NaN,NaN,NaN,NaN,NaN
3.875,NaN,10.0,3.0,2.0,1.0
3.750,NaN,NaN,NaN,NaN,1.0


In [4]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path = '../data/dot_plot_2024_12.csv'
)
data = loader.load()

출력은 다음과 같다. 객체가 `Document`인 것을 확인할 수 있다.

In [5]:
data

[Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 0}, page_content='Target rate: 4.750\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 1}, page_content='Target rate: 4.625\n2024: 4\n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 2}, page_content='Target rate: 4.500\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 3}, page_content='Target rate: 4.375\n2024: 15\n2025: 1\n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 4}, page_content='Target rate: 4.250\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'row': 5}, page_content='Target rate: 4.125\n2024: \n2025: 3\n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../data/dot_plot_2024_12.csv', 'ro

#### Document Transformation

문서를 가져온 후에는 필요에 더 잘 맞게 수정하는 것이 일반적이다. 기본적인 예로, 긴 문서를 모델의 문맥 창에 맞는 작은 chunk로 나누는 것이 있다. 랭체인에는 text splitters라고 하는 다양한 사전 구축된 문서 변환기가 존재한다. 텍스트 분할기는 문맥 및 관련 정보를 보존하면서 문서를 의미론적 연관성이 있는 청크로 더 쉽게 분할한다.

텍스트 분할기를 사용하면 텍스트를 분할하는 방법과 청크의 길이를 측정하는 방법을 결정할 수 있다. 예를 들어, 글자(character) 수준에서 작동하는 `RecursiveCharacterTextSplitter` module을 사용하여 문서를 분할해 보자.

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open('../data/Artificial Intelligence in the Financial System.txt') as file :
    text_file = file.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

texts = text_splitter.create_documents([text_file])

여기서 `chunk_size`는 각 청크의 글자 수를 나타내고 `chunk_overlap`은 연속된 청크 간에 겹치는 글자 수를 나타낸다. 출력은 다음과 같다

In [7]:
print(texts[0])
print(texts[1])
print(texts[2])

page_content='Discussions of artificial intelligence (AI) inevitably center on two main points: risks and'
page_content='points: risks and benefits.1 Both of these can be frustratingly vague and amorphous. Proponents of'
page_content='Proponents of AI project its widespread adoption will be as momentous as the industrial'


#### Text Embedding Model

embedding은 nonparametic 지식을 LLM에 통합하는 핵심 단계이다. 실제로 embedding이 vector DB에 제대로 저장되면 사용자 쿼리의 거리를 측정할 수 있는 비모수적 지식이 된다.

임베딩을 시작하려면 임베딩 모델이 필요한데, 랭체인은 비모수적 지식과 사용자 쿼리의 임베딩을 각각 처리하는 두 가지 주요 모듈로 구성된 `Embedding` 클래스를 제공한다.

In [8]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [9]:
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embedding_model.embed_documents(
    [
        "Good morning!",
        "Good afternoon!",
        'Oh, hello!',
        'I want to report an accident',
        'Sorry to hear that, May I ask your name?',
        'Sure, Mario Rossi.',
    ]
)

print('<embedded documents>')
print(
    f"Number of vector : {len(embeddings)} \nDimension of vector : {len(embeddings[0])}"
)

<embedded documents>
Number of vector : 6 
Dimension of vector : 1536


문서 질의가 모두 임베딩된 후에 할 일은 두 요소 간의 유사도를 계산하고 문서 임베딩에서 가장 적합한 정보를 검색하는 것이다.

In [10]:
embedded_query = embedding_model.embed_query(
    "What was the name mentioned in the conversation?"
)

In [11]:
print('<questionare>')
print(f"Dimension of ter vector : {len(embeddings[0])}")
print(f"Sample of the first 5 elements of the vector : {embedded_query[:5]}")

<questionare>
Dimension of ter vector : 1536
Sample of the first 5 elements of the vector : [-0.010684116743505001, -0.010173137299716473, -0.0019674645736813545, 0.023056013509631157, -0.02686513401567936]


#### Vector Database

Vector DB는 임베딩을 사용하여 텍스트, 이미지, 오디오 또는 비디오와 같은 비정형 데이터를 저장하고 검색할 수 있는 데이터베이스의 한 유형이다. 임베딩을 사용하면 벡터 저장소는 빠르고 정확한 유사도 검색, 즉 주어진 쿼리에 가장 관련성이 높은 데이터를 사용할 수 있다. 여기서는 고밀도 벡터의 효율적인 유사도 검색 및 클러스터링을 위해 META AI Research에서 개발한 FAISS Vector storage를 사용해 보자.

In [12]:
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [15]:
raw_documents = TextLoader('../data/Artificial Intelligence in the Financial System.txt').load()
text_splitter = CharacterTextSplitter(
    chunk_size = 1100,
    chunk_overlap = 0,
    separator = '\n'
)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

여기서 `OpenAIEmbeddings()` 호출 시 사이즈에 따라서 토큰 비용이 부과되니 과금에 주의한다. 이제 비모수적 지식을 임베딩하고 저장했으니, 코사인 유사도를 측정값으로 사용하여 가장 유사한 텍스트 청크를 검색하는 데 사용할 수 있도록 사용자의 쿼리도 임베딩해 보자

In [17]:
query = "How does AI affect the financial system?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Looking at the financial industry-specific implications of AI, it is helpful to consider not only how it may change the financial system, but also how regulatory frameworks should respond to this emerging technology. Are the existing frameworks sufficient? If not, how can regulators best balance the risks AI may pose to bank safety and soundness and financial stability with the need to allow for continued innovation?
Broader availability of generative AI and large language models have created headlines and spiking stock prices, but the financial services sector has been using AI for some time.2 Over time, it has become clear that AI's impact could be far-reaching, particularly as the technology becomes more efficient, new sources of data become available, and as AI technology becomes more affordable.
Do We Need a Definition of AI?


출력은 질문에 대한 답을 포함할 가능성이 더 높은 텍스트 조각이다. 엔드 투 엔드 시나리오에서는 대화형 응답을 생성하기 위한 LLM의 문맥으로 사용된다.

#### Retrievers

검색기는 자연어 질문이나 키워드와 같은 비정형 쿼리와 관련된 문서를 반환할 수 있는 랭체인의 구성 요소이다. 검색기는 문서 자체를 저장할 필요 없이 source에서 문서를 검색하기만 하면 된다. 검색기는 키워드 매칭, 의미론적 검색, 순위 알고리즘 등 다양한 방법을 사용해 관련 문서를 찾을 수 있다.

검색기와 벡터 저장소의 차이점은 검색기가 벡터 저장소보다 더 일반적이고 유연하다는 점이다. 검색기는 모든 방법을 사용해 관련 문서를 찾을 수 있는 반면, 벡터 저장소는 임베딩과 유사도 메트릭에 의존한다. 또한 검색기는 웹 페이지, 데이터베이스, 파일 등 다양한 문서 소스를 사용할 수 있는 반면, 벡터 저장소는 데이터 자체를 저장해야 한다.

그러나 데이터가 벡터 저장소에 의해 임베딩되고 색인된 경우, 벡터 저장소를 검색기의 백본으로 사용할 수도 있다. 이 경우 검색기는 벡터 저장소를 사용하여 임베딩된 데이터에 대해 유사도 검색을 수행하고 가장 관련성이 높은 문서를 반환할 수 있다. 이것은 랭체인의 주요 검색기 유형 중 하나이며, 벡터 저장소 검색기라고 불린다.

예를 들어서, 이전에 초기화했던 FAISS 벡터 저장소를 고려하고 그 위에 검색기를 마운팅해 보자.

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = db.as_retriever(search_kwargs={"k": 4})

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Use the context to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question: {question}
Answer:"""
)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
query = "How does AI affect the financial system?"
answer = rag_chain.invoke(query)

In [22]:
print(answer)

AI has the potential to reshape the financial services industry and the broader world economy by enhancing efficiency, reducing operational costs, improving fraud prevention, and providing better customer service. It can influence various aspects of the financial system, including the supervision of the payment system, banking operations, and monetary policy discussions. Additionally, AI's integration into the financial system raises questions about regulatory frameworks, competition, risks, and the balance between innovation and safety. Overall, while AI presents numerous benefits, it also introduces risks that need to be addressed within the regulatory landscape.
